[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/castorini/pygaggle/blob/master/notebooks/pygaggle_Colab_demo.ipynb)

# GPU runtime mode

The first step is to enable the GPU runtime mode. 
In order to change runtime mode(Runtime -> Change Runtime type -> python3 + GPU)

After this, please check your GPU accessibility.


In [ ]:
# Check nvidia and nvcc cuda compiler

!nvidia-smi
!/usr/local/cuda/bin/nvcc --version

Tue Sep  8 22:35:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Mount Google Drive

This step is optional but recommended. Mounting Google drive is helpful to manage files.

In [ ]:
# link to google drive

from google.colab import drive
drive.mount('/content/gdrive/')


#Setup SSH port forwarding

[Ngrok](https://ngrok.com/) is used to make SSH port forwarding. Hence, a Google Colab is transformed into a real server with SSH.

An account is required for Ngrok. Code below is used to generate a username and password.

In [ ]:
#Generate a random root password
import random, string
password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(30))


#Setup sshd
! apt-get install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen > /dev/null

#Set root password
! echo root:$password | chpasswd
! mkdir -p /var/run/sshd
! echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
! echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config

print("username: root")
print("password: ", password)

#Run sshd
get_ipython().system_raw('/usr/sbin/sshd -D &')


Creating config file /etc/ssh/sshd_config with new version
Creating SSH2 RSA key; this may take some time ...
2048 SHA256:QNqllSXD7oFQsnEPtlIk/k290+ysmGtV/3e7DmazdBI root@f71194a89fb5 (RSA)
Creating SSH2 ECDSA key; this may take some time ...
256 SHA256:+pPtbXC1LTgWisgiO/hN/nfuPhGlkaikkRORExOjdgY root@f71194a89fb5 (ECDSA)
Creating SSH2 ED25519 key; this may take some time ...
256 SHA256:8/Yi9dhe5dfsK//VzF6DxbSD6TayNlJejS7Q5wNeCg4 root@f71194a89fb5 (ED25519)
Created symlink /etc/systemd/system/sshd.service → /lib/systemd/system/ssh.service.
Created symlink /etc/systemd/system/multi-user.target.wants/ssh.service → /lib/systemd/system/ssh.service.
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
username: root
password:  vpQYNOSeV3IS7kcH3QuNnndDOw5nv1


The password above is used to connect your server.

After this step, Ngrok is downloaded, and it creates a TCP channel.

In [ ]:
# Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

In [ ]:
# setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')

Get your authtoken from https://dashboard.ngrok.com/auth
··········


# Connect the Colab instance

Now, a Colab instance is transformed into a server. You can access your server through ssh command in your local machine. 

Port number can be found through Ngrok interface https://dashboard.ngrok.com/status.



```
ssh root@0.tcp.ngrok.io -p [port_number]

> then enter the password generated previously in phase "Setup SSH port forwarding".

```

# Neural Ranking Baselines on MS MARCO Passage Retrieval

Now, you can run various neural reranking baselines on the MS MARCO passage ranking tasks. Please follow [instructions](https://github.com/castorini/pygaggle/blob/master/docs/experiments-msmarco-passage.md) here.

After you finish your tasks, please kill Ngrok.

In [ ]:
# When done, kill Ngrok

!kill $(ps aux | grep './ngrok' | awk '{print $2}')

^C
